In [47]:
import pandas as pd 

In [48]:
itinerarios_bases=pd.read_parquet('../Data/Itinerarios/itinerarios_bases3.parquet')
itinerarios_bases.head()

,user_type,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,status,Origen_destino,Número_Salida,Distrito_Salida,...,Barrio_Llegada,Número de Plazas_Llegada,Latitud_Llegada,Longitud_Llegada,name_Salida,address_Salida,name_Llegada,address_Llegada,dayofweek,hour
0,1,154,44,5,40,2019-07-01 00:00:00+00:00,short_rental,44-40,44,01 CENTRO,...,01-02 EMBAJADORES,24.0,40.413788,-3.704746,Plaza de Nelson Mandela,Calle Mesón de Paredes nº 35,Conde de Romanones,Plaza del Conde de Romanones nº 9,0,0
1,1,184,57,4,129,2019-07-01 00:00:00+00:00,short_rental,57-129,57,01 CENTRO,...,07-05 RÍOS ROSAS,24.0,40.438469,-3.698282,Santa Cruz del Marcenado,Calle Santa Cruz del Marcenado nº 24,José Abascal,Calle José Abascal nº 33,0,0
2,1,282,53,4,43,2019-07-01 00:00:00+00:00,short_rental,53-43,53,01 CENTRO,...,01-02 EMBAJADORES,24.0,40.410088,-3.702545,Plaza de Lavapiés,Calle Valencia nº 1,Jesús y María,Calle Lavapiés nº 34-36,0,0
3,1,388,56,0,13,2019-07-01 00:00:00+00:00,short_rental,56-13,56,01 CENTRO,...,01-05 UNIVERSIDAD,24.0,40.427364,-3.710473,Plaza del Carmen,Plaza del Carmen nº 1,Conde Duque,Calle Conde Duque nº 22,0,0
4,1,254,105,0,75,2019-07-01 00:00:00+00:00,short_rental,105-75,105,04 SALAMANCA,...,03-05 JERÓNIMOS,24.0,40.408340,-3.678415,Serrano,Calle Serrano nº 54,Menéndez Pelayo,Avenida de Menéndez Pelayo nº 90,0,0


In [49]:
itinerarios_bases.columns

Index(['user_type', 'travel_time', 'idunplug_station', 'ageRange',
       'idplug_station', 'unplug_hourTime', 'status', 'Origen_destino',
       'Número_Salida', 'Distrito_Salida', 'Barrio_Salida',
       'Número de Plazas_Salida', 'Latitud_Salida', 'Longitud_Salida',
       'Número_Llegada', 'Distrito_Llegada', 'Barrio_Llegada',
       'Número de Plazas_Llegada', 'Latitud_Llegada', 'Longitud_Llegada',
       'name_Salida', 'address_Salida', 'name_Llegada', 'address_Llegada',
       'dayofweek', 'hour'],
      dtype='object')

In [50]:
def DistrubicionEstaciones(itinerarios_bases): 
    cols_grup=['idplug_station','Latitud_Llegada', 'Longitud_Llegada', 'dayofweek', 'hour']
    df_llegada = itinerarios_bases.groupby(cols_grup)['status'].count().to_frame().reset_index()
    df_llegada.rename(columns={'status': 'viajes_llegada', 'idplug_station': 'estacion'}, inplace=True)
    # Percentage by lambda and DataFrame.apply() method.

    df3 = df_llegada.groupby(['dayofweek', 'hour'])['viajes_llegada'].sum().to_dict()#.reset_index()
    df_llegada['dict']=df_llegada.apply(lambda x: (x.dayofweek,x.hour),axis=1)
    df_llegada['viajes_llegada%']=df_llegada.apply(lambda x: x.viajes_llegada/df3[x.dict], axis=1)
    df_llegada_def=df_llegada[['estacion','Latitud_Llegada', 'Longitud_Llegada','dayofweek', 'hour', 'viajes_llegada', 'viajes_llegada%' ]]
    df_llegada_def.rename(columns={'Latitud_Llegada':'Latitud','Longitud_Llegada':'Longitud' }, inplace=True)


    cols_grup2=['idunplug_station','Latitud_Salida', 'Longitud_Salida', 'dayofweek', 'hour']
    df_salida = itinerarios_bases.groupby(cols_grup2)['status'].count().to_frame().reset_index()
    df_salida.rename(columns={'status': 'viajes_salida', 'idunplug_station': 'estacion'}, inplace=True)
    # Percentage by lambda and DataFrame.apply() method.

    df3 = df_salida.groupby(['dayofweek', 'hour'])['viajes_salida'].sum().to_dict()#.reset_index()
    df_salida['dict']=df_salida.apply(lambda x: (x.dayofweek,x.hour),axis=1)
    df_salida['viajes_salida%']=df_salida.apply(lambda x: x.viajes_salida/df3[x.dict], axis=1)
    df_salida_def=df_salida[['estacion','Latitud_Salida', 'Longitud_Salida','dayofweek', 'hour', 'viajes_salida', 'viajes_salida%' ]]
    
    df_salida_def.rename(columns={'Latitud_Salida':'Latitud','Longitud_Salida':'Longitud' }, inplace=True)

    return pd.merge(df_llegada_def,df_salida_def, how='inner', on=['estacion','dayofweek', 'hour', 'Latitud', 'Longitud'])
a=DistrubicionEstaciones(itinerarios_bases)
a

c:\Users\visit\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,estacion,Latitud,Longitud,dayofweek,hour,viajes_llegada,viajes_llegada%,viajes_salida,viajes_salida%
0,1,40.417110,-3.701998,0,0,8,0.002851,33,0.011761
1,1,40.417110,-3.701998,0,1,7,0.003989,20,0.011396
2,1,40.417110,-3.701998,0,2,10,0.008354,17,0.014202
3,1,40.417110,-3.701998,0,3,20,0.018832,11,0.010358
4,1,40.417110,-3.701998,0,4,30,0.013743,9,0.004123
...,...,...,...,...,...,...,...,...,...
34368,219,40.408398,-3.646778,6,19,42,0.004241,47,0.004746
34369,219,40.408398,-3.646778,6,20,22,0.002779,31,0.003916
34370,219,40.408398,-3.646778,6,21,27,0.003796,33,0.004640
34371,219,40.408398,-3.646778,6,22,17,0.002789,35,0.005742


In [51]:
def mapaPrediccion(itinerarios_bases, prediccion, salida_llegada,dia, hora ): 
    dicSalidaLlegada={0:'prediccion_llegada', 1:'prediccion_salida' }
    distribuciones=DistrubicionEstaciones(itinerarios_bases)
    distribuciones_filt=distribuciones[(distribuciones['dayofweek']==dia)&(distribuciones['hour']==hora)]

    distribuciones_filt['prediccion_llegada']= distribuciones_filt['viajes_llegada%']*prediccion
    distribuciones_filt['prediccion_salida']= distribuciones_filt['viajes_salida%']*prediccion

    fig = px.scatter_mapbox(distribuciones_filt, lat="Latitud", lon="Longitud",   size=dicSalidaLlegada[salida_llegada], 
                            zoom = 12, color_discrete_sequence=px.colors.qualitative.Prism, )
                #color_discrete_sequence=[to_hex(c) for c in sns.color_palette('colorblind', 15)])
    # fig.update_traces(hovertemplate=None)

    fig.update_layout(
        
        #autosize=True,
        showlegend=True,

        height = 650,
        mapbox=dict(
        bearing=0,
        center=dict(
            lat=40.435,
            lon=-3.69
        ),
        zoom=11.5,
        style= 'carto-positron' # 'open-street-map'
        ),
        legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1),
        margin=dict(l=1,r=1, b=10,t=20,pad = 5),
        hoverlabel_align = 'right'
        )
    return fig

In [52]:
import plotly.express as px
prediccion= 1200
salida_llegada=0
day=0
hora=7

dicSalidaLlegada={0:'prediccion_llegada', 1:'prediccion_salida' }
filt=a[(a['dayofweek']==day)&(a['hour']==hora)]

filt['prediccion_llegada']= filt['viajes_llegada%']*prediccion
filt['prediccion_salida']= filt['viajes_salida%']*prediccion

fig = px.scatter_mapbox(filt, lat="Latitud", lon="Longitud",   size=dicSalidaLlegada[salida_llegada], 
                        zoom = 12, color_discrete_sequence=px.colors.qualitative.Prism, )
            #color_discrete_sequence=[to_hex(c) for c in sns.color_palette('colorblind', 15)])
# fig.update_traces(hovertemplate=None)

fig.update_layout(
    
    #autosize=True,
    showlegend=True,

    height = 650,
    mapbox=dict(
    bearing=0,
    center=dict(
        lat=40.435,
        lon=-3.69
    ),
    zoom=11.5,
    style= 'carto-positron' # 'open-street-map'
    ),
    legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1),
    margin=dict(l=1,r=1, b=10,t=20,pad = 5),
    hoverlabel_align = 'right'
    )
fig.show()

<ipython-input-52-bec9f23d50ba>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-52-bec9f23d50ba>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

